In [1]:
import networkx as nx

In [6]:
def pagerank(G, alpha=0.85, personalization=None, max_iter=100, tol=1.0e-6, nstart=None, weight='weight', dangling=None):
    if len(G) == 0:
        return {}

    if not G.is_directed():
        D = G.to_directed()
    else:
        D = G

    # Create a copy in (right) stochastic form
    W = nx.stochastic_graph(D, weight=weight)
    N = W.number_of_nodes()

    # Choose fixed starting vector if not given
    if nstart is None:
        x = dict.fromkeys(W, 1.0 / N)
    else:
        # Normalized nstart vector
        s = float(sum(nstart.values()))
        x = dict((k, v / s) for k, v in nstart.items())

    if personalization is None:

        # Assign uniform personalization vector if not given
        p = dict.fromkeys(W, 1.0 / N)
    else:
        missing = set(G) - set(personalization)
        if missing:
            raise NetworkXError('Personalization dictionary '
                                'must have a value for every node. '
                                'Missing nodes %s' % missing)
        s = float(sum(personalization.values()))
        p = dict((k, v / s) for k, v in personalization.items())

    if dangling is None:

        # Use personalization vector if dangling vector not specified
        dangling_weights = p
    else:
        missing = set(G) - set(dangling)
        if missing:
            raise NetworkXError('Dangling node dictionary '
                                'must have a value for every node. '
                                'Missing nodes %s' % missing)
        s = float(sum(dangling.values()))
        dangling_weights = dict((k, v/s) for k, v in dangling.items())
    dangling_nodes = [n for n in W if W.out_degree(n, weight=weight) == 0.0]

    # power iteration: make up to max_iter iterations
    for _ in range(max_iter):
        xlast = x
        x = dict.fromkeys(xlast.keys(), 0)
        danglesum = alpha * sum(xlast[n] for n in dangling_nodes)
        for n in x:

        # this matrix multiply looks odd because it is
        # doing a left multiply x^T=xlast^T*W
            for nbr in W[n]:
                x[nbr] += alpha * xlast[n] * W[n][nbr][weight]
            x[n] += danglesum * dangling_weights[n] + (1.0 - alpha) * p[n]

        # check convergence, l1 norm
        err = sum([abs(x[n] - xlast[n]) for n in x])
        if err < N*tol:
            return x
    raise NetworkXError('pagerank: power iteration failed to converge '
                        'in %d iterations.' % max_iter)


In [7]:
G=nx.barabasi_albert_graph(60,41)
pr=nx.pagerank(G,0.4)
pr

{0: 0.01218054803977264,
 1: 0.013161914595643425,
 2: 0.012974526077350654,
 3: 0.013378166539094028,
 4: 0.012775277367604854,
 5: 0.013370155413106028,
 6: 0.012972269382777528,
 7: 0.012765966908339066,
 8: 0.012961626872192471,
 9: 0.013167560003931466,
 10: 0.013167571163738482,
 11: 0.013376307484919572,
 12: 0.013153879539029823,
 13: 0.01258372719949036,
 14: 0.01256698911434037,
 15: 0.012963776558164462,
 16: 0.012547210747293659,
 17: 0.012758845985909456,
 18: 0.012753808148281337,
 19: 0.012769009314739179,
 20: 0.012762353979625773,
 21: 0.012968512274405824,
 22: 0.012752970746269528,
 23: 0.012561829728928458,
 24: 0.013372432488901983,
 25: 0.01297306057783693,
 26: 0.012949797368214945,
 27: 0.012748641092662091,
 28: 0.013163088118442608,
 29: 0.013358796637580753,
 30: 0.012973559336139156,
 31: 0.011948961015038841,
 32: 0.013174663794122488,
 33: 0.013153879539029823,
 34: 0.012745960927697529,
 35: 0.0127644929687635,
 36: 0.0133597416076249,
 37: 0.012754303220